In [41]:
import torch

# v1 is a dict
v1 = {
    "1": torch.tensor([0.0, 1.0,1.0], requires_grad=True)  # We will overwrite v1["1"][0] with `a` each iteration
}

# Target norm
target = torch.tensor(5.0)

# Trainable parameter a (1-element tensor)
a = torch.nn.Parameter(torch.tensor(0.0))

# Fixed part of the vector
v1_rest = torch.tensor([1.0, 1.0])  # The fixed second and third components

# Fixed vector to cross with
v2 = torch.tensor([2.0, 1.0, 1.0])  # shape: (3,)

# Optimizer
optimizer = torch.optim.SGD([a], lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()

    # Dynamically create v1 with current a
    v1["1"] = torch.cat([a.view(1), v1_rest])  # shape: (3,)

    # Compute cross product
    cross = torch.cross( v1["1"], v2)

    # Compute norm
    norm = torch.norm(cross)

    # Loss function: squared error
    loss = (norm - target) ** 2

    # Backprop and optimization
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: a = {a.item():.4f}, norm = {norm.item():.4f}, loss = {loss.item():.6f}")

print(f"\nFinal value of a: {a.item():.4f}")



Epoch 0: a = -0.0614, norm = 2.8284, loss = 4.715729
Epoch 10: a = -0.5555, norm = 3.5563, loss = 2.084364
Epoch 20: a = -0.8840, norm = 4.0402, loss = 0.921294
Epoch 30: a = -1.1024, norm = 4.3619, loss = 0.407214
Epoch 40: a = -1.2475, norm = 4.5757, loss = 0.179990
Epoch 50: a = -1.3441, norm = 4.7179, loss = 0.079556
Epoch 60: a = -1.4082, norm = 4.8125, loss = 0.035164
Epoch 70: a = -1.4509, norm = 4.8753, loss = 0.015543
Epoch 80: a = -1.4793, norm = 4.9171, loss = 0.006870
Epoch 90: a = -1.4981, norm = 4.9449, loss = 0.003036

Final value of a: -1.5096
